#### SOLO PROFILO MEDIO:


In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from IPython.display import display


cleaned_buildings = gpd.read_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/cluster_with_buildings.geojson"
)

# Calcolo della densità abitativa su tutta la shape.
house = cleaned_buildings[cleaned_buildings["tags_building"] == "house"]
area_tot = house["area_m2"].sum()
pop_microgrid = (
    93641.59  # Qua devi usare la funzione nel codice per calcolare la popolazione.
)
density = pop_microgrid / area_tot

# Calcolo della house-area per ogni cluster.
grouped_buildings = cleaned_buildings.groupby("cluster_id")
clusters = np.sort(cleaned_buildings["cluster_id"].unique())
house_area_for_cluster = []
for cluster in clusters:
    cluster_buildings = pd.DataFrame(grouped_buildings.get_group(cluster))
    house = cluster_buildings[cluster_buildings["tags_building"] == "house"]
    area_house = house["area_m2"].sum()
    house_area_for_cluster.append(area_house)

# Creazione di un DataFrame con la popolazione suddivisa nei cluster sulla base della densità-area_house
population_df = pd.DataFrame()
population_df["cluster"] = clusters
population_df.set_index("cluster", inplace=True)
population_df["house_area_for_cluster"] = house_area_for_cluster
people_for_cluster = (population_df["house_area_for_cluster"] * density).round()
population_df["people_for_cluster"] = people_for_cluster

display(population_df)

,house_area_for_cluster,people_for_cluster
cluster,,
0,167199.174127,7850.0
1,198012.745308,9297.0
2,235799.960999,11071.0
3,233431.361591,10960.0
4,246815.535349,11588.0
5,252566.604783,11858.0
6,231201.805873,10855.0
7,60611.022440,2846.0
8,196700.620729,9235.0


In [2]:
# Creazione di un DataFrame con la popolazione per ogni cluster suddivisa nei vari Tier
tier_percent = [0.3, 0.2, 0.2, 0.1, 0.15, 0.05]
people_for_cluster = population_df["people_for_cluster"]
tier_pop_df = population_df["people_for_cluster"].apply(
    lambda x: pd.Series([x * y for y in tier_percent])
)

display(tier_pop_df)

,0,1,2,3,4,5
cluster,,,,,,
0,2355.0,1570.0,1570.0,785.0,1177.50,392.50
1,2789.1,1859.4,1859.4,929.7,1394.55,464.85
2,3321.3,2214.2,2214.2,1107.1,1660.65,553.55
3,3288.0,2192.0,2192.0,1096.0,1644.00,548.00
4,3476.4,2317.6,2317.6,1158.8,1738.20,579.40
5,3557.4,2371.6,2371.6,1185.8,1778.70,592.90
6,3256.5,2171.0,2171.0,1085.5,1628.25,542.75
7,853.8,569.2,569.2,284.6,426.90,142.30
8,2770.5,1847.0,1847.0,923.5,1385.25,461.75


In [3]:
# Importazione dei profili medi e della deviazione standard per ogni Tier.
demand_tier_1 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier1.xlsx"
)
demand_tier_2 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier2.xlsx"
)
demand_tier_3 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier3.xlsx"
)
demand_tier_4 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier4.xlsx"
)
demand_tier_5 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier5.xlsx"
)

# Creazione di un DataFrame con tutti i tier e la domanda media oraria per ognuno
mean_demand_tier_df = pd.DataFrame()
demand_tiers = [
    demand_tier_1,
    demand_tier_2,
    demand_tier_3,
    demand_tier_4,
    demand_tier_5,
]

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"bus_{i}"
    mean_demand_tier_df[mean_column_name] = demand_tier["mean"]
mean_demand_tier_df.insert(0, "bus_0", np.zeros(len(mean_demand_tier_df)))

hours_index = pd.date_range(
    start="00:00:00", periods=len(mean_demand_tier_df), freq="H", normalize=True
)
mean_demand_tier_df.index = hours_index.time

# Creazione di un DataFrame con tutti i tier e la std media oraria per ognuno
std_demand_tier_df = pd.DataFrame()

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"bus_{i}"
    std_demand_tier_df[mean_column_name] = demand_tier["std"]
std_demand_tier_df.insert(0, "bus_0", np.zeros(len(mean_demand_tier_df)))

std_demand_tier_df.index = hours_index.time

display(mean_demand_tier_df)
display(std_demand_tier_df)

C:\Users\margh\AppData\Local\Temp\ipykernel_12332\3033006911.py:33: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours_index = pd.date_range(


,bus_0,bus_1,bus_2,bus_3,bus_4,bus_5
00:00:00,0.0,0.000000,0.000000,28.991322,42.427125,42.278934
01:00:00,0.0,0.000000,0.000000,34.521576,50.553214,50.452805
02:00:00,0.0,0.000000,0.000000,36.611001,53.669763,53.635554
03:00:00,0.0,0.000000,0.000000,38.924761,57.097983,57.000288
04:00:00,0.0,0.000032,0.046572,42.550663,62.418339,62.367183
05:00:00,0.0,0.138339,6.277810,60.382878,90.390240,90.930738
06:00:00,0.0,0.596456,31.498172,102.802546,161.313978,164.726412
07:00:00,0.0,0.960828,46.257291,133.108190,237.445624,240.651913
08:00:00,0.0,0.632438,27.723440,116.696750,213.293060,216.459444
09:00:00,0.0,0.171015,9.780120,87.361677,147.039771,162.527548


,bus_0,bus_1,bus_2,bus_3,bus_4,bus_5
00:00:00,0.0,0.000000,0.000000,0.865204,1.403410,1.424645
01:00:00,0.0,0.000000,0.000000,0.990713,1.454768,1.445946
02:00:00,0.0,0.000000,0.000000,1.057676,1.773486,1.669474
03:00:00,0.0,0.000000,0.000000,1.245038,1.784916,1.793179
04:00:00,0.0,0.000193,0.055545,1.426176,2.050683,1.988202
05:00:00,0.0,0.048147,1.236822,2.257312,3.834563,4.090113
06:00:00,0.0,0.095504,2.652671,4.242210,7.589726,7.596358
07:00:00,0.0,0.110751,3.165970,5.152397,13.336545,12.446711
08:00:00,0.0,0.096623,2.712762,4.803722,12.682301,12.466618
09:00:00,0.0,0.049724,1.583367,3.643800,6.890035,10.493887


In [4]:
import numpy as np
import pandas as pd

# Creo un dizionario di DataFrame ( uno per ogni coppia tier cluster) ogni colonna del dataframe è il carico giornaliero(orario) di una persona,
# rappresentato dal valore medio

result_dict = {}
for k in range(len(tier_pop_df)):
    pop_cluster = tier_pop_df.iloc[k, :]
    for j in range(len(pop_cluster)):
        num_iterations = int(pop_cluster[j])
        num_rows = len(mean_demand_tier_df)
        num_columns = num_iterations
        loads = np.zeros((num_rows, num_columns))
        for i in range(num_iterations):
            mean_load_person = mean_demand_tier_df.iloc[:, j].values
            load_person = mean_load_person
            loads[:, i] = load_person
        results_df = pd.DataFrame(loads)

        nome_dataframe = f"person_load_cluster_{k}_tier_{j}"
        result_dict[nome_dataframe] = results_df


result_dict["person_load_cluster_0_tier_1"]

,0,1,2,3,4,5,6,7,8,9,...,1560,1561,1562,1563,1564,1565,1566,1567,1568,1569
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,...,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032,0.000032
5,0.138339,0.138339,0.138339,0.138339,0.138339,0.138339,0.138339,0.138339,0.138339,0.138339,...,0.138339,0.138339,0.138339,0.138339,0.138339,0.138339,0.138339,0.138339,0.138339,0.138339
6,0.596456,0.596456,0.596456,0.596456,0.596456,0.596456,0.596456,0.596456,0.596456,0.596456,...,0.596456,0.596456,0.596456,0.596456,0.596456,0.596456,0.596456,0.596456,0.596456,0.596456
7,0.960828,0.960828,0.960828,0.960828,0.960828,0.960828,0.960828,0.960828,0.960828,0.960828,...,0.960828,0.960828,0.960828,0.960828,0.960828,0.960828,0.960828,0.960828,0.960828,0.960828
8,0.632438,0.632438,0.632438,0.632438,0.632438,0.632438,0.632438,0.632438,0.632438,0.632438,...,0.632438,0.632438,0.632438,0.632438,0.632438,0.632438,0.632438,0.632438,0.632438,0.632438
9,0.171015,0.171015,0.171015,0.171015,0.171015,0.171015,0.171015,0.171015,0.171015,0.171015,...,0.171015,0.171015,0.171015,0.171015,0.171015,0.171015,0.171015,0.171015,0.171015,0.171015


In [5]:
#  Faccio un DataFrame con i dati cumulati di ogni coppia bus-tier associando a ogni riga il risultato della sommatoria sulle righe.
total_loads_cluster_tier_dict = {}

for k in range(len(tier_pop_df)):
    key_cluster = f"{k}"
    total_loads_cluster_tier_dict[key_cluster] = {}

    for j in range(len(pop_cluster)):
        key_tier = f"{j}"
        load = result_dict[f"person_load_cluster_{key_cluster}_tier_{key_tier}"]
        load_tot = load.sum(axis=1)
        total_loads_df = pd.DataFrame(
            load_tot, columns=[f"bus_{key_cluster}_tier_{key_tier}"]
        )

        total_loads_cluster_tier_dict[key_cluster][key_tier] = total_loads_df

total_loads_cluster_tier_dict["0"]["1"]

,bus_0_tier_1
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.050787
5,217.192605
6,936.436623
7,1508.499523
8,992.927575
9,268.493140


In [6]:
# Sommo lungo le righe, per ogni cluster, tutti i dataframe relativi ai vari tier di quel cluster per ottener eun carioc unico del cluster
total_loads_cluster = {}

for k in range(len(tier_pop_df)):
    key_cluster = str(k)
    df0 = total_loads_cluster_tier_dict[key_cluster]["0"].values
    df1 = total_loads_cluster_tier_dict[key_cluster]["1"].values
    df2 = total_loads_cluster_tier_dict[key_cluster]["2"].values
    df3 = total_loads_cluster_tier_dict[key_cluster]["3"].values
    df4 = total_loads_cluster_tier_dict[key_cluster]["4"].values
    df5 = total_loads_cluster_tier_dict[key_cluster]["5"].values

    total_load = pd.DataFrame(df0 + df1 + df2 + df3 + df4 + df5, columns=[f"bus_{k}"])
    total_loads_cluster[f"bus_{k}"] = total_load

# Creo un DataFRame unico per ogni cluster.
tot_loads_df = pd.DataFrame()
for key, cluster_load in total_loads_cluster.items():
    tot_loads_df = pd.concat([tot_loads_df, cluster_load], axis=1)

tot_loads_df

,bus_0,bus_1,bus_2,bus_3,bus_4,bus_5,bus_6,bus_7,bus_8,bus_9
0,89268.256376,105693.776182,1.259027e+05,1.246935e+05,1.317898e+05,1.348193e+05,1.234421e+05,32311.099454,105011.147321,91927.353376
1,106378.069835,125951.826220,1.500341e+05,1.485933e+05,1.570496e+05,1.606597e+05,1.471020e+05,38504.095144,125138.359422,109546.862240
2,112934.083999,133714.166601,1.592806e+05,1.577510e+05,1.667286e+05,1.705611e+05,1.561678e+05,40877.091988,132850.566066,116298.205375
3,120104.376115,142203.824717,1.693935e+05,1.677668e+05,1.773143e+05,1.813902e+05,1.660830e+05,43472.413721,141285.393441,123682.078474
4,131389.761348,155565.742460,1.853103e+05,1.835307e+05,1.939753e+05,1.984342e+05,1.816887e+05,47557.259047,154561.012620,135303.618498
5,199508.074339,236219.170938,2.813817e+05,2.786754e+05,2.945367e+05,3.013112e+05,2.758847e+05,72217.932953,234693.575513,205446.850626
6,385527.871538,456472.220794,5.437348e+05,5.384933e+05,5.691489e+05,5.822517e+05,5.331191e+05,139568.672020,453524.264934,396991.389198
7,552431.424403,654097.678128,7.791319e+05,7.716265e+05,8.155616e+05,8.343316e+05,7.639277e+05,199994.242807,649873.445212,568863.366015
8,472023.710160,558892.564758,6.657318e+05,6.593293e+05,6.968688e+05,7.128965e+05,6.527387e+05,170876.455930,555283.097856,486074.876205
9,320978.806656,380044.379733,4.527049e+05,4.483598e+05,4.738802e+05,4.847707e+05,4.438620e+05,116190.765975,377589.855476,330542.599968


### CON STD

In [7]:
import geopandas as gpd
import pandas as pd
import numpy as np
from IPython.display import display

cleaned_buildings = gpd.read_file(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/cluster_with_buildings.geojson"
)

# Calcolo della densità abitativa su tutta la shape.
house = cleaned_buildings[cleaned_buildings["tags_building"] == "house"]
area_tot = house["area_m2"].sum()
pop_microgrid = (
    93641.59  # Qua devi usare la funzione nel codice per calcolare la popolazione.
)
density = pop_microgrid / area_tot

# Calcolo della house-area per ogni cluster.
grouped_buildings = cleaned_buildings.groupby("cluster_id")
clusters = np.sort(cleaned_buildings["cluster_id"].unique())
house_area_for_cluster = []
for cluster in clusters:
    cluster_buildings = pd.DataFrame(grouped_buildings.get_group(cluster))
    house = cluster_buildings[cluster_buildings["tags_building"] == "house"]
    area_house = house["area_m2"].sum()
    house_area_for_cluster.append(area_house)

# Creazione di un DataFrame con la popolazione suddivisa nei cluster sulla base della densità-area_house
population_df = pd.DataFrame()
population_df["cluster"] = clusters
population_df.set_index("cluster", inplace=True)
population_df["house_area_for_cluster"] = house_area_for_cluster
people_for_cluster = (population_df["house_area_for_cluster"] * density).round()
population_df["people_for_cluster"] = people_for_cluster

display(population_df)

,house_area_for_cluster,people_for_cluster
cluster,,
0,167199.174127,7850.0
1,198012.745308,9297.0
2,235799.960999,11071.0
3,233431.361591,10960.0
4,246815.535349,11588.0
5,252566.604783,11858.0
6,231201.805873,10855.0
7,60611.022440,2846.0
8,196700.620729,9235.0


In [8]:
# Creazione di un DataFrame con la popolazione per ogni cluster suddivisa nei vari Tier
tier_percent = [0.3, 0.2, 0.2, 0.1, 0.15, 0.05]
people_for_cluster = population_df["people_for_cluster"]
tier_pop_df = population_df["people_for_cluster"].apply(
    lambda x: pd.Series([x * y for y in tier_percent])
)

display(tier_pop_df)

,0,1,2,3,4,5
cluster,,,,,,
0,2355.0,1570.0,1570.0,785.0,1177.50,392.50
1,2789.1,1859.4,1859.4,929.7,1394.55,464.85
2,3321.3,2214.2,2214.2,1107.1,1660.65,553.55
3,3288.0,2192.0,2192.0,1096.0,1644.00,548.00
4,3476.4,2317.6,2317.6,1158.8,1738.20,579.40
5,3557.4,2371.6,2371.6,1185.8,1778.70,592.90
6,3256.5,2171.0,2171.0,1085.5,1628.25,542.75
7,853.8,569.2,569.2,284.6,426.90,142.30
8,2770.5,1847.0,1847.0,923.5,1385.25,461.75


In [9]:
# Importazione dei profili medi e della deviazione standard per ogni Tier.
demand_tier_1 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier1.xlsx"
)
demand_tier_2 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier2.xlsx"
)
demand_tier_3 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier3.xlsx"
)
demand_tier_4 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier4.xlsx"
)
demand_tier_5 = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/ramp/daily_type_demand_Tier5.xlsx"
)

# Creazione di un DataFrame con tutti i tier e la domanda media oraria per ognuno
mean_demand_tier_df = pd.DataFrame()
demand_tiers = [
    demand_tier_1,
    demand_tier_2,
    demand_tier_3,
    demand_tier_4,
    demand_tier_5,
]

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"bus_{i}"
    mean_demand_tier_df[mean_column_name] = demand_tier["mean"]
mean_demand_tier_df.insert(0, "bus_0", np.zeros(len(mean_demand_tier_df)))

hours_index = pd.date_range(
    start="00:00:00", periods=len(mean_demand_tier_df), freq="H", normalize=True
)
mean_demand_tier_df.index = hours_index.time

# Creazione di un DataFrame con tutti i tier e la std media oraria per ognuno
std_demand_tier_df = pd.DataFrame()

for i, demand_tier in enumerate(demand_tiers, start=1):
    mean_column_name = f"bus_{i}"
    std_demand_tier_df[mean_column_name] = demand_tier["std"]
std_demand_tier_df.insert(0, "bus_0", np.zeros(len(mean_demand_tier_df)))

std_demand_tier_df.index = hours_index.time

display(mean_demand_tier_df)
display(std_demand_tier_df)

C:\Users\margh\AppData\Local\Temp\ipykernel_12332\3033006911.py:33: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hours_index = pd.date_range(


,bus_0,bus_1,bus_2,bus_3,bus_4,bus_5
00:00:00,0.0,0.000000,0.000000,28.991322,42.427125,42.278934
01:00:00,0.0,0.000000,0.000000,34.521576,50.553214,50.452805
02:00:00,0.0,0.000000,0.000000,36.611001,53.669763,53.635554
03:00:00,0.0,0.000000,0.000000,38.924761,57.097983,57.000288
04:00:00,0.0,0.000032,0.046572,42.550663,62.418339,62.367183
05:00:00,0.0,0.138339,6.277810,60.382878,90.390240,90.930738
06:00:00,0.0,0.596456,31.498172,102.802546,161.313978,164.726412
07:00:00,0.0,0.960828,46.257291,133.108190,237.445624,240.651913
08:00:00,0.0,0.632438,27.723440,116.696750,213.293060,216.459444
09:00:00,0.0,0.171015,9.780120,87.361677,147.039771,162.527548


,bus_0,bus_1,bus_2,bus_3,bus_4,bus_5
00:00:00,0.0,0.000000,0.000000,0.865204,1.403410,1.424645
01:00:00,0.0,0.000000,0.000000,0.990713,1.454768,1.445946
02:00:00,0.0,0.000000,0.000000,1.057676,1.773486,1.669474
03:00:00,0.0,0.000000,0.000000,1.245038,1.784916,1.793179
04:00:00,0.0,0.000193,0.055545,1.426176,2.050683,1.988202
05:00:00,0.0,0.048147,1.236822,2.257312,3.834563,4.090113
06:00:00,0.0,0.095504,2.652671,4.242210,7.589726,7.596358
07:00:00,0.0,0.110751,3.165970,5.152397,13.336545,12.446711
08:00:00,0.0,0.096623,2.712762,4.803722,12.682301,12.466618
09:00:00,0.0,0.049724,1.583367,3.643800,6.890035,10.493887


In [10]:
import numpy as np
import pandas as pd


result_dict = {}

for k in range(len(tier_pop_df)):
    pop_cluster = tier_pop_df.iloc[k, :]
    for j in range(len(pop_cluster)):
        num_iterations = int(pop_cluster[j])
        num_rows = len(mean_demand_tier_df)
        num_columns = num_iterations
        loads = np.zeros((num_rows, num_columns))
        for i in range(num_iterations):
            mean_load_person = mean_demand_tier_df.iloc[:, j].values
            std_load_person = np.random.normal(
                0, mean_demand_tier_df.iloc[:, j].values[:num_rows]
            )
            load_person = mean_load_person + std_load_person
            loads[:, i] = load_person
        results_df = pd.DataFrame(loads)

        nome_dataframe = f"person_load_cluster_{k}_tier_{j}"
        result_dict[nome_dataframe] = results_df


result_dict["person_load_cluster_0_tier_1"]

,0,1,2,3,4,5,6,7,8,9,...,1560,1561,1562,1563,1564,1565,1566,1567,1568,1569
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
4,0.000025,0.000071,-0.000003,-0.000012,0.000081,-0.000018,0.000011,0.000079,0.000007,0.000046,...,0.000060,-7.139280e-07,0.000046,0.000035,-0.000007,-0.000004,0.000009,0.000058,1.110767e-04,0.000011
5,0.217141,0.237935,0.174233,-0.016584,-0.075075,0.110636,0.062452,-0.198969,0.068507,0.228773,...,0.175090,1.072757e-01,0.103430,0.018845,0.334769,-0.017013,0.426977,0.237003,7.344750e-02,0.066650
6,0.939072,0.422109,0.275598,0.399452,0.660856,0.618188,0.012545,1.599381,0.072003,-0.294878,...,0.499554,8.752231e-01,0.802658,0.378417,1.210443,0.305815,-0.213234,0.735098,7.965065e-01,0.641653
7,1.633466,-0.491798,0.674120,1.661171,0.769262,1.220812,0.303457,1.224325,0.399829,0.344052,...,1.283208,2.528529e+00,-0.083167,1.367685,1.712408,0.504942,-0.462136,1.920327,1.007276e+00,2.110558
8,0.481599,1.431093,0.461544,0.644469,1.284563,-0.103409,0.490392,0.272925,0.691221,0.547744,...,0.696789,1.915479e-01,0.177414,1.620566,0.203944,-0.308277,0.728710,0.472468,1.096344e-01,0.692918
9,0.186697,0.278571,0.192430,0.284590,0.065272,0.198870,0.170431,0.284941,0.042803,0.157265,...,0.061752,5.514548e-02,0.183488,0.254755,0.240351,0.226633,0.301253,0.292216,2.197946e-01,0.047553


In [11]:
total_loads_cluster_tier_dict = {}

# Iterating over each cluster (key_cluster)
for k in range(len(tier_pop_df)):
    key_cluster = f"{k}"
    total_loads_cluster_tier_dict[key_cluster] = (
        {}
    )  # Create a nested dictionary for the current key_cluster

    # Iterating over each tier (key_tier)
    for j in range(len(pop_cluster)):
        key_tier = f"{j}"
        load = result_dict[f"person_load_cluster_{key_cluster}_tier_{key_tier}"]
        load_tot = load.sum(axis=1)
        total_loads_df = pd.DataFrame(
            load_tot, columns=[f"bus_{key_cluster}_tier_{key_tier}"]
        )

        # Store the dataframe in the nested dictionary
        total_loads_cluster_tier_dict[key_cluster][key_tier] = total_loads_df

total_loads_cluster_tier_dict["0"]["1"]

,bus_0_tier_1
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.048698
5,213.929714
6,923.935033
7,1576.180579
8,987.684956
9,267.420131


In [15]:
total_loads_cluster = {}

for k in range(len(tier_pop_df)):
    key_cluster = str(k)
    df0 = total_loads_cluster_tier_dict[key_cluster]["0"].values
    df1 = total_loads_cluster_tier_dict[key_cluster]["1"].values
    df2 = total_loads_cluster_tier_dict[key_cluster]["2"].values
    df3 = total_loads_cluster_tier_dict[key_cluster]["3"].values
    df4 = total_loads_cluster_tier_dict[key_cluster]["4"].values
    df5 = total_loads_cluster_tier_dict[key_cluster]["5"].values

    total_load = pd.DataFrame(df0 + df1 + df2 + df3 + df4 + df5, columns=[f"bus_{k}"])
    total_loads_cluster[f"bus_{k}"] = total_load

tot_loads_df = pd.DataFrame()
for key, cluster_load in total_loads_cluster.items():
    tot_loads_df = pd.concat([tot_loads_df, cluster_load], axis=1)

tot_loads_df

,bus_0,bus_1,bus_2,bus_3,bus_4,bus_5,bus_6,bus_7,bus_8,bus_9
0,89572.356959,102954.893599,1.244788e+05,1.263605e+05,1.339282e+05,1.388575e+05,1.214362e+05,32556.951437,105914.411646,93611.940450
1,104532.557737,127633.922712,1.503380e+05,1.464852e+05,1.588193e+05,1.621242e+05,1.447333e+05,39478.885705,128635.626783,111046.185282
2,108578.532844,134511.005228,1.566454e+05,1.549807e+05,1.646894e+05,1.707079e+05,1.514091e+05,40796.800509,130732.007202,117846.018857
3,120481.045694,138517.981314,1.721839e+05,1.696769e+05,1.756384e+05,1.810397e+05,1.627788e+05,42175.150590,139447.105961,125805.893260
4,131869.468413,158308.288303,1.878493e+05,1.868255e+05,1.969221e+05,1.988049e+05,1.865026e+05,47780.730584,153243.060287,136778.946071
5,197746.887340,240448.548824,2.858999e+05,2.830228e+05,2.943941e+05,3.089477e+05,2.747518e+05,70446.065858,240049.860131,206351.339806
6,387316.356334,461575.393861,5.244259e+05,5.412797e+05,5.571132e+05,5.760938e+05,5.423145e+05,142137.419176,471222.079619,386681.500233
7,548535.194297,658137.587860,7.630818e+05,7.675609e+05,8.007045e+05,8.399287e+05,7.927242e+05,200128.211221,665302.418805,563017.021624
8,478895.481160,557855.514314,6.596881e+05,6.593867e+05,6.771390e+05,7.151059e+05,6.581784e+05,165886.014663,554776.671048,468142.662222
9,326864.902423,386908.459206,4.691687e+05,4.544872e+05,4.862620e+05,4.841408e+05,4.398614e+05,118234.955883,366022.633030,331641.859427


In [13]:
date_range = pd.date_range(start="2013-01-01", end="2013-12-31", freq="D")
yearly_mean_demand_tier_df = pd.concat(
    [tot_loads_df] * len(date_range), ignore_index=True
)
date_time_index = pd.date_range(start="2013-01-01", end="2013-12-31 23:00:00", freq="H")
yearly_mean_demand_tier_df.index = date_time_index

display(yearly_mean_demand_tier_df)

C:\Users\margh\AppData\Local\Temp\ipykernel_12332\2013365005.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_time_index = pd.date_range(start="2013-01-01", end="2013-12-31 23:00:00", freq="H")


,bus_0,bus_1,bus_2,bus_3,bus_4,bus_5,bus_6,bus_7,bus_8,bus_9
2013-01-01 00:00:00,89572.356959,102954.893599,1.244788e+05,1.263605e+05,1.339282e+05,1.388575e+05,1.214362e+05,32556.951437,105914.411646,93611.940450
2013-01-01 01:00:00,104532.557737,127633.922712,1.503380e+05,1.464852e+05,1.588193e+05,1.621242e+05,1.447333e+05,39478.885705,128635.626783,111046.185282
2013-01-01 02:00:00,108578.532844,134511.005228,1.566454e+05,1.549807e+05,1.646894e+05,1.707079e+05,1.514091e+05,40796.800509,130732.007202,117846.018857
2013-01-01 03:00:00,120481.045694,138517.981314,1.721839e+05,1.696769e+05,1.756384e+05,1.810397e+05,1.627788e+05,42175.150590,139447.105961,125805.893260
2013-01-01 04:00:00,131869.468413,158308.288303,1.878493e+05,1.868255e+05,1.969221e+05,1.988049e+05,1.865026e+05,47780.730584,153243.060287,136778.946071
...,...,...,...,...,...,...,...,...,...,...
2013-12-31 19:00:00,788578.154280,949998.214756,1.151826e+06,1.117126e+06,1.221414e+06,1.217133e+06,1.096765e+06,308434.764336,977773.439477,854797.104348
2013-12-31 20:00:00,668196.664839,776322.111545,9.319835e+05,9.028743e+05,9.665206e+05,9.771386e+05,9.152871e+05,231493.597740,753713.583295,686909.306769
2013-12-31 21:00:00,497084.256831,570537.636522,6.942478e+05,6.958852e+05,7.322072e+05,7.345169e+05,6.678383e+05,182409.425888,573538.301639,506846.222411
2013-12-31 22:00:00,307155.146881,344876.376133,4.280240e+05,4.260845e+05,4.495964e+05,4.629803e+05,4.201621e+05,113043.394099,354653.946018,321365.337967
